# Default of credit card clients Data Set
### Cristian Ferney Caicedo
### Christian David Cardona
### Jonathan Smith Llanos

In [ ]:
#cargar datos desde drive acceso libre
FILEID = "1O3viIXyAB9E1z09hyQqPw8HtcF5QQt4H"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O codigos.zip && rm -rf /tmp/cookies.txt
!unzip codigos.zip
!dir

--2022-02-22 14:26:53--  https://docs.google.com/uc?export=download&confirm=&id=1O3viIXyAB9E1z09hyQqPw8HtcF5QQt4H
Resolving docs.google.com (docs.google.com)... 108.177.121.113, 108.177.121.101, 108.177.121.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.121.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/612pbj613e7loj4ns1cvme4vdd41knmd/1645539975000/04733746419931218404/*/1O3viIXyAB9E1z09hyQqPw8HtcF5QQt4H?e=download [following]
--2022-02-22 14:26:54--  https://doc-0o-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/612pbj613e7loj4ns1cvme4vdd41knmd/1645539975000/04733746419931218404/*/1O3viIXyAB9E1z09hyQqPw8HtcF5QQt4H?e=download
Resolving doc-0o-28-docs.googleusercontent.com (doc-0o-28-docs.googleusercontent.com)... 108.177.120.132, 2607:f8b0:4001:c18::84
Connecting to doc-0o-28-docs.googleusercontent.com (doc-0

In [ ]:
# Importamos las librerias:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras

import pandas as pd
%matplotlib inline
csv_path = 'default of credit card clients.xls'
Xdata = pd.read_excel(csv_path)

col_rename = ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
              'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 
              'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'Default Payment Next Mont']
Xdata.columns = col_rename
Xdata.drop(index = 0, columns="ID", inplace = True)
Xdata.head()

ImportError: ignored

# Verificamos datos del DataFrame:

In [ ]:
Xdata.info()

Convertimos los valores tipo objeto del DataFrame a enteros

In [ ]:
Xdata=Xdata.astype('int')
Xdata.info()

# Analisis exploratorio:

### Histograma de todos los atributos:

In [ ]:
Xdata2 = Xdata.drop(columns="LIMIT_BAL")

In [ ]:
Xdata.hist(layout=(6,4),figsize=(20,20))
plt.show()

### Analisis exploratorio:

### Analizaremos el monto de credito otorgado por educación:

In [ ]:
plt.figure(figsize=(12,7), dpi=110)
plt.subplot(221)
Xdata["LIMIT_BAL"].groupby(Xdata["EDUCATION"]).sum().plot(kind='pie', cmap='Paired')
plt.title("Diagarama de pastel")

plt.subplot(222)
Xdata.groupby("EDUCATION")["LIMIT_BAL"].sum().plot(kind='bar', legend='Reverse',)
plt.title("Histograma")

plt.show()

Donde:

**1** indica doctorado

**2** indica pregrado

**3** indica bachillerato

**4 en adelante** otros

*Vemos que a las personas con doctorado se les hizo un préstamo mayor que el resto*

### Analizaremos el monto de credito otorgado por género:

In [ ]:
plt.figure(figsize=(12,7), dpi=110)
plt.subplot(221)
Xdata["LIMIT_BAL"].groupby(Xdata["SEX"]).sum().plot(kind='pie', cmap='Paired')
plt.title("Diagarama de pastel")

plt.subplot(222)
Xdata.groupby("SEX")["LIMIT_BAL"].sum().plot(kind='bar', legend='Reverse',)
plt.title("Histograma")

plt.show()

Indicando que **'1' equivale a los hombres** y **'2' a las mujeres**

*Vemos que a las mujeres se le hizo un prestamo mayor que a los hombres*

In [ ]:
genero = np.array(Xdata["SEX"])
count2 = np.zeros(4)
# count2[0] es hombre negado
# count2[1] es hombre aceptado
# count2[3] es mujer negada
# count2[4] es mujer aceptada
for i in range (30000):
  if (genero[i]==1):
    count2[0] = count2[0]+1
  if (genero[i]==2):
    count2[1] = count2[1]+1

print(count2[0])
print(count2[1])
total = count2[0]+count2[1]
hombres = count2[0]/total*100
mujeres = count2[1]/total*100
print(hombres, mujeres)

In [ ]:
probabilidad_m = [hombres, mujeres]
nombres3 =["MEN = 39.6%", "WOMEN = 60.37%"]

plt.figure(figsize=(7, 5), dpi=120)
plt.pie(probabilidad_m, labels=nombres3)
plt.title("QUANTITY MEN VS WOMEN")

### Calculamos la probabilidad de credibilidad de clientes:

---



In [ ]:
y_train2 = np.array(Xdata["Default Payment Next Mont"])
y_train2.shape

In [ ]:
i=0
count = np.zeros(2)
for i in range (30000):
  if y_train2[i]==0:
    count[0] = count[0]+1
  else:
    count[1] = count[1]+1
  i = i+1

total = count[0]+count[1]
si = count[1]/total*100
no = count[0]/total*100
print("La probabilidad de clientes no creibles es:",no, "La probabilidad de clientes creibles es:", si)

### Calculamos la probabilidad de credibilidad a clientes por incumpliento de pago:

#### Tomando en cuenta el género:

In [ ]:
genero = np.array(Xdata["SEX"])
count2 = np.zeros(4)
# count2[0] es hombre negado
# count2[1] es hombre aceptado
# count2[3] es mujer negada
# count2[4] es mujer aceptada
for i in range (30000):
  if (genero[i]==1 and y_train2[i]==0):
    count2[0] = count2[0]+1
  if (genero[i]==1 and y_train2[i]==1):
    count2[1] = count2[1]+1
  if (genero[i]==2 and y_train2[i]==0):
    count2[2] = count2[2]+1
  if (genero[i]==2 and y_train2[i]==1):
    count2[3] = count2[3]+1

total_h = count2[0]+count2[1]
total_m = count2[2]+count2[3]
hombre_no = count2[0]/total_h*100
hombre_si = count2[1]/total_h*100
mujer_no = count2[2]/total_m*100
mujer_si = count2[3]/total_m*100

print("La probabilidad de clientes hombres no creibles es:",hombre_no, "La probabilidad de hombres creibles es", hombre_si)
print("La probabilidad de mujeres no creibles es:",mujer_no, "La probabilidad de mujeres creibles es:",mujer_si)

In [ ]:
probabilidad = [no,si]
nombres = ["Not credible = 78%","Credible =22%"]

probabilidad_h = [hombre_no, hombre_si]
nombres2 =["Not credible = 89%", "Credible = 11%"]

probabilidad_m = [mujer_no, mujer_si]
nombres3 =["Not credible = 79%", "Credible = 21%"]

plt.figure(figsize=(10, 8), dpi=110)
plt.subplot(221)
plt.pie(probabilidad, labels=nombres)
plt.title("Customer credibility")

plt.subplot(333)
plt.pie(probabilidad_h, labels=nombres2)
plt.title("Credibility of male clients")

plt.subplot(336)
plt.pie(probabilidad_m, labels=nombres3)
plt.title("Credibility of female clients")

plt.show()

#### Tomando en cuenta la educación:

In [ ]:
educacion = np.array(Xdata["EDUCATION"])
count3 = np.zeros(6)
# count3[0] es Doctorado creible
# count3[1] es Doctorado no creible
# count3[2] es pregrado creible
# count3[3] es pregrado no creible
# count3[4] es bachillerato creible
# count3[5] es bacchillerato no creible
for i in range (30000):
  if (educacion[i]==1 and y_train2[i]==0):
    count3[0] = count3[0]+1
  if (educacion[i]==1 and y_train2[i]==1):
    count3[1] = count3[1]+1
  if (educacion[i]==2 and y_train2[i]==0):
    count3[2] = count3[2]+1
  if (educacion[i]==2 and y_train2[i]==1):
    count3[3] = count3[3]+1
  if (educacion[i]==3 and y_train2[i]==1):
    count3[4] = count3[4]+1
  if (educacion[i]==3 and y_train2[i]==1):
    count3[5] = count3[5]+1

total_d = count3[0]+count3[1]
total_p = count3[2]+count3[3]
total_b = count3[4]+count3[5]

doctorado_no = count3[0]/total_d*100
doctorado_si = count3[1]/total_d*100

pregrado_no = count3[2]/total_p*100
pregrado_si = count3[3]/total_p*100

bachillerato_no = count3[4]/total_b*100
bachillerato_si = count3[5]/total_b*100

print("La probabilidad de clientes con doctorado no creibles es:",doctorado_no, "La probabilidad de clientes con doctorado creibles es", doctorado_si)
print("La probabilidad de clientes con pregrado no creibles es:",pregrado_no, "La probabilidad de clientes con pregrado creibles es:",pregrado_si)
print("La probabilidad de clientes con bachillerato no creibles es:",bachillerato_no, "La probabilidad de clientes con bachillerato creibles es:",bachillerato_si)

In [ ]:
probabilidad_d = [doctorado_no, doctorado_si]
nombres4 =["Not credible = 81%", "Credible = 19%"]

probabilidad_p = [pregrado_no, pregrado_si]
nombres5 =["Not credible = 76%", "Credible = 24%"]

probabilidad_b = [bachillerato_no, bachillerato_si]
nombres6 =["Not credible = 50%", "Credible = 50%"]

plt.figure(figsize=(10, 8), dpi=110)
plt.subplot(331)
plt.pie(probabilidad_d, labels=nombres4)
plt.title("Credibility of clients with PhD")

plt.subplot(333)
plt.pie(probabilidad_p, labels=nombres5)
plt.title("Credibility of clients with professional title")

plt.subplot(335)
plt.pie(probabilidad_b, labels=nombres6)
plt.title("Credibility of clients with a high school degree")

plt.show()

# Dividimos la base de datos en Entrenamiento, Validación y Test:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
salida = 'Default Payment Next Mont'
y_data = Xdata[salida]
X_data = Xdata.drop(columns='Default Payment Next Mont')
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X_data, y_data):
    X_train_full = X_data.iloc[train_index]
    X_test = X_data.iloc[test_index]
    y_train_full = y_data.iloc[train_index]
    y_test = y_data.iloc[test_index]
    
for train_index, valid_index in split.split(X_train_full, y_train_full):
    X_train = X_train_full.iloc[train_index]
    X_valid = X_train_full.iloc[valid_index]
    y_train = y_train_full.iloc[train_index]
    y_valid = y_train_full.iloc[valid_index]   

In [ ]:
import seaborn as sns
plt.figure(figsize=(8, 5), dpi=80)
sns.distplot(y_train, bins=10, label="Indicando 0 como 'no' y 1 como 'si'")
plt.title("Histograma de credibilidad de clientes")
plt.legend()
plt.show()

### Diagrama de dispersión:

In [ ]:
from sklearn.decomposition import PCA
zpca = PCA(n_components=2).fit_transform(X_train)
zpca.shape

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(zpca[:,0],zpca[:,1],c=y_train, alpha=0.8)
plt.title("SCATTER OF DATABASE")
plt.legend()
plt.colorbar()
plt.show()

## Matriz de correlación:

In [ ]:
corr_matrix = Xdata.corr()
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,7), dpi=100)
sns.heatmap(corr_matrix,xticklabels=corr_matrix.columns.values,yticklabels=corr_matrix.columns.values)
plt.tight_layout()
#plt.savefig('results/corr_matrix.pdf', format='pdf', dpi=300)
plt.show()

# Normalizamos con StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Realizamos modelo con Redes Neuronales Funcionales

In [ ]:
input_l = tf.keras.layers.Input(shape=(X_train.shape[1:]), name='entrada')
h1 = tf.keras.layers.Dense(300,activation='tanh',name='h1')(input_l)
h2 = tf.keras.layers.Dense(150,activation='tanh',name='h2')(h1)
h3 = tf.keras.layers.Dense(75,activation='tanh',name='h3')(h2)
h4 = tf.keras.layers.Dense(30,activation='tanh',name='h4')(h3)
output = tf.keras.layers.Dense(1,activation="sigmoid",name='output')(h4)
model_fun = tf.keras.Model(inputs=input_l,outputs=output)
model_fun.summary()
tf.keras.utils.plot_model(model_fun)

## Compilamos modelo:

In [ ]:
model_fun.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

## Entrenamos el modelo:

In [ ]:

history = model_fun.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))



### Graficamos la relación entre el loss y el accuracy


In [ ]:
history.history.keys()

In [ ]:
0

### Evaluamos el modelo:

In [ ]:
model_fun.evaluate(X_test, y_test)

# Realizamos las predicciones:

In [ ]:
defi = model_fun.predict(X_test) #output
defi.shape

## Curva ROC:

In [ ]:
from sklearn.metrics import roc_curve
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, defi)
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(figsize=(7,4), dpi=110)
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.grid()
plt.show()

## Matriz de confusion

In [ ]:
def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = 100*cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
          
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:

defi_b = (defi>=0.5).astype("int").ravel()
no = np.array(['No confiable'])
si = np.array(['Confiable'])
a = np.concatenate((no,si))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels

acc = accuracy_score(y_test, defi_b)
plot_confusion_matrix(y_test, defi_b, normalize=True, title='ACC = %.1f %%' % (100*acc))
plt.autoscale()                  
plt.show()

## Analisis exploratorio PCA por capas:

In [ ]:
model_B = tf.keras.Model(inputs=model_fun.inputs,outputs=model_fun.get_layer('h1').output) #se accede a capas por nombre
tf.keras.utils.plot_model(model_B) # modleo ya esta ajustado!
zb = model_B.predict(X_train) # salida en h4
zpcb = PCA(n_components=2).fit_transform(zb)

model_C = tf.keras.Model(inputs=model_fun.inputs,outputs=model_fun.get_layer('h2').output) #se accede a capas por nombre
tf.keras.utils.plot_model(model_C) # modleo ya esta ajustado!
zc = model_C.predict(X_train) # salida en h4
zpcc = PCA(n_components=2).fit_transform(zc)

model_D = tf.keras.Model(inputs=model_fun.inputs,outputs=model_fun.get_layer('h3').output) #se accede a capas por nombre
tf.keras.utils.plot_model(model_D) # modleo ya esta ajustado!
zd = model_D.predict(X_train) # salida en h4
zpcd = PCA(n_components=2).fit_transform(zd)

model_E = tf.keras.Model(inputs=model_fun.inputs,outputs=model_fun.get_layer('h4').output) #se accede a capas por nombre
tf.keras.utils.plot_model(model_E) # modleo ya esta ajustado!
ze = model_E.predict(X_train) # salida en h4
zpce = PCA(n_components=2).fit_transform(ze)

plt.figure(figsize=(15,8))

plt.subplot(221)
plt.scatter(zpcb[:,0],zpcb[:,1],c=y_train)
plt.colorbar()

plt.subplot(222)
plt.scatter(zpcc[:,0],zpcb[:,1],c=y_train)
plt.colorbar()

plt.subplot(223)
plt.scatter(zpcd[:,0],zpcd[:,1],c=y_train)
plt.colorbar()

plt.subplot(224)
plt.scatter(zpce[:,0],zpce[:,1],c=y_train)
plt.colorbar()
plt.show()

In [ ]:
X_train.shape

In [ ]:
X_train_full.shape

In [ ]:
print(X_data.shape)
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

In [ ]:
print(y_data.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)